## Installing demoji

In [1]:
!pip install demoji

## Imports

In [2]:
import pandas as pd
import numpy as np

import keras

from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## Loading Datasets

In [3]:
df_train = pd.read_excel('/content/drive/MyDrive/Datasets/Malayalam__hasoc_train.xlsx',names=["ID","Tweets","Labels"])
df_train.dropna(inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_val = pd.read_csv("/content/drive/MyDrive/Datasets/malayalam_hasoc_dev.tsv", sep="\t",names=["ID","Tweets","Labels"])
df_val.dropna(inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [4]:
df_train_Tweet = pd.DataFrame(df_train.Tweets)
df_val_Tweet = pd.DataFrame(df_val.Tweets)
df_train_Label = pd.DataFrame(df_train.Labels)
df_val_Label = pd.DataFrame(df_val.Labels)


## Preprocessing

In [5]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))
import demoji

def preprocessing(document):
        document = demoji.replace_with_desc(document).replace(":"," ").replace("-"," ")
        document = regex.sub(' ', document)
        document = re.sub(r'[0-9]', '', document)
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)             # remove all single characters
        document = re.sub(r'\s+', ' ', document, flags=re.I)            # Substituting multiple spaces with single space
        return document
    


In [6]:
corpus_train = df_train_Tweet.Tweets.apply(preprocessing)
corpus_val = df_val_Tweet.Tweets.apply(preprocessing)

## TF-IDF Vectorization

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(1,5))
cv = TfidfVectorizer(max_features=15000)
X_train = cv.fit_transform(corpus_train).toarray()
X_test = cv.transform(corpus_val).toarray()
X_train.shape

(3999, 15000)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Label Encoding

In [9]:
Y_train = np.ravel(df_train_Label)
Y_test = np.ravel(df_val_Label)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y_train)
Y_train = le.transform(Y_train)
Y_test = le.transform(Y_test)

#Model Creation and Validation

## SIMPLE ANN

In [10]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(512, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(256, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(128, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               7680512   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 7,844,865
Trainable params: 7,844,865
Non-trainable params: 0
_________________________________________________________________


In [12]:
model_save_path='/content/m1'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [13]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=30,
                    callbacks=callbacks)

Epoch 1/30
134/134 [==============================] - 13s 9ms/step - loss: 0.6930 - accuracy: 0.5082 - val_loss: 0.6923 - val_accuracy: 0.5615
Epoch 2/30
134/134 [==============================] - 1s 6ms/step - loss: 0.6862 - accuracy: 0.7814 - val_loss: 0.6898 - val_accuracy: 0.6309
Epoch 3/30
134/134 [==============================] - 1s 5ms/step - loss: 0.6717 - accuracy: 0.9172 - val_loss: 0.6799 - val_accuracy: 0.6824
Epoch 4/30
134/134 [==============================] - 1s 6ms/step - loss: 0.6245 - accuracy: 0.9533 - val_loss: 0.6431 - val_accuracy: 0.6898
Epoch 5/30
134/134 [==============================] - 1s 5ms/step - loss: 0.4736 - accuracy: 0.9711 - val_loss: 0.5873 - val_accuracy: 0.7045
Epoch 6/30
134/134 [==============================] - 1s 5ms/step - loss: 0.2490 - accuracy: 0.9843 - val_loss: 0.5895 - val_accuracy: 0.7045
Epoch 7/30
134/134 [==============================] - 1s 5ms/step - loss: 0.1104 - accuracy: 0.9943 - val_loss: 0.6276 - val_accuracy: 0.7066
Epoch

In [14]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.69      0.75      0.72       473
         OFF       0.73      0.67      0.70       478

    accuracy                           0.71       951
   macro avg       0.71      0.71      0.71       951
weighted avg       0.71      0.71      0.71       951



###Creating padded sequences

In [15]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(corpus_train)

X_train = tokenizer.texts_to_sequences(corpus_train)
X_test = tokenizer.texts_to_sequences(corpus_val)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(corpus_train[2])
print(X_train[2])

fukru nem tiktok oolakale vilich charcha nadathiyenekka bedham aanh
[985, 2416, 1238, 1641, 787, 257, 4302, 2417, 4303]


In [16]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 120

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## ANN with Embedding matrix

In [17]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [18]:
optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 100)          1916700   
_________________________________________________________________
flatten (Flatten)            (None, 12000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               6144512   
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 8,225,565
Trainable params: 8,225,565
Non-trainable params: 0
____________________________________________

In [19]:
model_save_path='/content/m2'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [20]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 4s 22ms/step - loss: 0.6967 - accuracy: 0.5028 - val_loss: 0.6976 - val_accuracy: 0.5026
Epoch 2/30
125/125 [==============================] - 2s 20ms/step - loss: 0.6925 - accuracy: 0.5195 - val_loss: 0.6928 - val_accuracy: 0.5026
Epoch 3/30
125/125 [==============================] - 2s 20ms/step - loss: 0.5441 - accuracy: 0.7625 - val_loss: 0.6786 - val_accuracy: 0.6856
Epoch 4/30
125/125 [==============================] - 2s 19ms/step - loss: 0.1164 - accuracy: 0.9602 - val_loss: 0.8548 - val_accuracy: 0.6751
Epoch 5/30
125/125 [==============================] - 2s 19ms/step - loss: 0.0534 - accuracy: 0.9752 - val_loss: 1.0281 - val_accuracy: 0.6635
Epoch 6/30
125/125 [==============================] - 2s 20ms/step - loss: 0.0219 - accuracy: 0.9899 - val_loss: 1.1723 - val_accuracy: 0.6635
Epoch 7/30
125/125 [==============================] - 3s 21ms/step - loss: 0.0167 - accuracy: 0.9945 - val_loss: 1.1565 - val_accuracy: 0.6614

In [21]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.66      0.77      0.71       473
         OFF       0.72      0.60      0.66       478

    accuracy                           0.69       951
   macro avg       0.69      0.69      0.68       951
weighted avg       0.69      0.69      0.68       951



####Reduced dimension to 50

In [22]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 50)           958350    
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 958,871
Trainable params: 958,871
Non-trainable params: 0
_________________________________________________________________


In [23]:
model_save_path='/content/m3'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [24]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 2s 13ms/step - loss: 0.6924 - accuracy: 0.5169 - val_loss: 0.6833 - val_accuracy: 0.6088
Epoch 2/30
125/125 [==============================] - 1s 11ms/step - loss: 0.6555 - accuracy: 0.7823 - val_loss: 0.6118 - val_accuracy: 0.7161
Epoch 3/30
125/125 [==============================] - 1s 12ms/step - loss: 0.4652 - accuracy: 0.8978 - val_loss: 0.5523 - val_accuracy: 0.7382
Epoch 4/30
125/125 [==============================] - 2s 12ms/step - loss: 0.2472 - accuracy: 0.9360 - val_loss: 0.5811 - val_accuracy: 0.7266
Epoch 5/30
125/125 [==============================] - 1s 12ms/step - loss: 0.1300 - accuracy: 0.9686 - val_loss: 0.6274 - val_accuracy: 0.7213
Epoch 6/30
125/125 [==============================] - 1s 12ms/step - loss: 0.0649 - accuracy: 0.9877 - val_loss: 0.6788 - val_accuracy: 0.7108
Epoch 7/30
125/125 [==============================] - 1s 12ms/step - loss: 0.0394 - accuracy: 0.9922 - val_loss: 0.7224 - val_accuracy: 0.7014

In [25]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.70      0.82      0.76       473
         OFF       0.79      0.65      0.72       478

    accuracy                           0.74       951
   macro avg       0.75      0.74      0.74       951
weighted avg       0.75      0.74      0.74       951



##CNN

In [26]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(layers.Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 100)          1916700   
_________________________________________________________________
conv1d (Conv1D)              (None, 120, 64)           19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [27]:
model_save_path='/content/m4'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [28]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 6s 25ms/step - loss: 0.6927 - accuracy: 0.5095 - val_loss: 0.6917 - val_accuracy: 0.4984
Epoch 2/30
125/125 [==============================] - 2s 19ms/step - loss: 0.6847 - accuracy: 0.5926 - val_loss: 0.6590 - val_accuracy: 0.6635
Epoch 3/30
125/125 [==============================] - 2s 19ms/step - loss: 0.5432 - accuracy: 0.8186 - val_loss: 0.5813 - val_accuracy: 0.6993
Epoch 4/30
125/125 [==============================] - 2s 19ms/step - loss: 0.2248 - accuracy: 0.9202 - val_loss: 0.6849 - val_accuracy: 0.6951
Epoch 5/30
125/125 [==============================] - 2s 19ms/step - loss: 0.1033 - accuracy: 0.9621 - val_loss: 0.8216 - val_accuracy: 0.6919
Epoch 6/30
125/125 [==============================] - 2s 18ms/step - loss: 0.0569 - accuracy: 0.9848 - val_loss: 0.9235 - val_accuracy: 0.6919
Epoch 7/30
125/125 [==============================] - 2s 18ms/step - loss: 0.0376 - accuracy: 0.9875 - val_loss: 1.0362 - val_accuracy: 0.6930

In [29]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.68      0.75      0.71       473
         OFF       0.73      0.64      0.68       478

    accuracy                           0.70       951
   macro avg       0.70      0.70      0.70       951
weighted avg       0.70      0.70      0.70       951



In [30]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(layers.Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 50)           958350    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 120, 64)           9664      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_14 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                

In [31]:
model_save_path='/content/m5'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [32]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 3s 14ms/step - loss: 0.6930 - accuracy: 0.5129 - val_loss: 0.6925 - val_accuracy: 0.4984
Epoch 2/30
125/125 [==============================] - 1s 10ms/step - loss: 0.6845 - accuracy: 0.5683 - val_loss: 0.6688 - val_accuracy: 0.6193
Epoch 3/30
125/125 [==============================] - 1s 11ms/step - loss: 0.5848 - accuracy: 0.7730 - val_loss: 0.5802 - val_accuracy: 0.7171
Epoch 4/30
125/125 [==============================] - 1s 11ms/step - loss: 0.2771 - accuracy: 0.8994 - val_loss: 0.6301 - val_accuracy: 0.6877
Epoch 5/30
125/125 [==============================] - 1s 11ms/step - loss: 0.1436 - accuracy: 0.9511 - val_loss: 0.7541 - val_accuracy: 0.6793
Epoch 6/30
125/125 [==============================] - 1s 11ms/step - loss: 0.0872 - accuracy: 0.9701 - val_loss: 0.8924 - val_accuracy: 0.6919
Epoch 7/30
125/125 [==============================] - 1s 11ms/step - loss: 0.0556 - accuracy: 0.9774 - val_loss: 1.0119 - val_accuracy: 0.6845

In [33]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.68      0.83      0.74       473
         OFF       0.78      0.61      0.68       478

    accuracy                           0.72       951
   macro avg       0.73      0.72      0.71       951
weighted avg       0.73      0.72      0.71       951



## BILSTM

In [34]:
embedding_dim = 100
max_len = 120

model = Sequential()

model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)))

model.add(layers.GlobalMaxPooling1D())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(2, activation='softmax'))

model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 100)          1916700   
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 512)          731136    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 20)                10260     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [35]:
model_save_path='/content/m6'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [36]:
history = model.fit(X_train, Y_train,
                     epochs=50,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks = callbacks)

Epoch 1/50
125/125 [==============================] - 16s 44ms/step - loss: 0.7771 - accuracy: 0.4862 - val_loss: 0.7464 - val_accuracy: 0.5026
Epoch 2/50
125/125 [==============================] - 4s 34ms/step - loss: 0.7394 - accuracy: 0.4921 - val_loss: 0.7272 - val_accuracy: 0.5026
Epoch 3/50
125/125 [==============================] - 4s 36ms/step - loss: 0.7108 - accuracy: 0.4814 - val_loss: 0.7110 - val_accuracy: 0.5026
Epoch 4/50
125/125 [==============================] - 4s 34ms/step - loss: 0.6818 - accuracy: 0.4876 - val_loss: 0.6902 - val_accuracy: 0.5026
Epoch 5/50
125/125 [==============================] - 4s 34ms/step - loss: 0.6579 - accuracy: 0.4967 - val_loss: 0.6915 - val_accuracy: 0.5026
Epoch 6/50
125/125 [==============================] - 4s 35ms/step - loss: 0.6484 - accuracy: 0.4728 - val_loss: 0.6698 - val_accuracy: 0.5026
Epoch 7/50
125/125 [==============================] - 4s 35ms/step - loss: 0.6215 - accuracy: 0.4883 - val_loss: 0.6689 - val_accuracy: 0.502

In [37]:
model.load_weights(model_save_path)

class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.63      0.83      0.72       473
         OFF       0.76      0.53      0.62       478

    accuracy                           0.68       951
   macro avg       0.69      0.68      0.67       951
weighted avg       0.70      0.68      0.67       951



####Reduced dimension to 50

In [38]:
embedding_dim = 50
max_len = 120

model = Sequential()

model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)))

model.add(layers.GlobalMaxPooling1D())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(2, activation='softmax'))

model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 50)           958350    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120, 512)          628736    
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 20)                10260     
_________________________________________________________________
dropout_5 (Dropout)          (None, 20)               

In [39]:
model_save_path='/content/m7'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [40]:
history = model.fit(X_train, Y_train,
                     epochs=100,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/100
125/125 [==============================] - 8s 34ms/step - loss: 0.7905 - accuracy: 0.4823 - val_loss: 0.7470 - val_accuracy: 0.5026
Epoch 2/100
125/125 [==============================] - 3s 25ms/step - loss: 0.7449 - accuracy: 0.4820 - val_loss: 0.7294 - val_accuracy: 0.5026
Epoch 3/100
125/125 [==============================] - 3s 25ms/step - loss: 0.7302 - accuracy: 0.4931 - val_loss: 0.7514 - val_accuracy: 0.5026
Epoch 4/100
125/125 [==============================] - 3s 27ms/step - loss: 0.7073 - accuracy: 0.4794 - val_loss: 0.8168 - val_accuracy: 0.5026
Epoch 5/100
125/125 [==============================] - 4s 28ms/step - loss: 0.6632 - accuracy: 0.4920 - val_loss: 0.6868 - val_accuracy: 0.5026
Epoch 6/100
125/125 [==============================] - 4s 28ms/step - loss: 0.6221 - accuracy: 0.4907 - val_loss: 0.6660 - val_accuracy: 0.5026
Epoch 7/100
125/125 [==============================] - 4s 28ms/step - loss: 0.5990 - accuracy: 0.4868 - val_loss: 0.6774 - val_accuracy:

In [41]:
model.load_weights(model_save_path)
import keras
class_names = df_train.Labels.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

         NOT       0.64      0.84      0.72       473
         OFF       0.77      0.52      0.62       478

    accuracy                           0.68       951
   macro avg       0.70      0.68      0.67       951
weighted avg       0.70      0.68      0.67       951

